### Variables

In [ ]:
host = ''
port = "5432"
dbname = "db_demo_rh"
user = ""
password = ""
schema_name = ""
jdbc_url = f"jdbc:postgresql://{host}:{port}/{dbname}"
driver = "org.postgresql.Driver"
storage_path = "s3://storage/"

jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

In [ ]:
tables_df = spark.read.jdbc(url=jdbc_url, table="(SELECT table_name FROM information_schema.tables WHERE table_schema = 'public') AS tables", properties=jdbc_properties)
tables = [row.table_name for row in tables_df.collect()]


for table in tables:
    delta_path = storage_path + f"path/to/db_demo_rh/{table}"

    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS db_demo_rh.{table}
    USING delta
    LOCATION '{delta_path}'
    """)
    
    df = spark.read.jdbc(url=jdbc_url, table=table, properties=jdbc_properties)
    df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_path)